# Standard Precipitaion Index - N (SPI-N)

This notebook implements the SPI for multiple time scales. It's common to refer to SPI in terms of months accumulated, for example SPI-2 for 2 months, SPI-3 for 3 months, and so on. 

The SPI-N, must be calculated for every month, regardless the N value. The calculation will be simply:<br>
<b>` (Rain_m - Avg_m) / Std_m`, where  </b>

The `m` indicates the month considered. So the `Avg_m` and `Std_m` can be precalculated for the 12 months, in advance. After that, even if we are calculating the SPI_m for for hundreds of months, the calculation will be pretty straightforward. 

The `Avg_m` for 1 month time scale is aready pre-calculated in the `MONTHLY_ACCUMULATED` data type. So, we need to pre-calculate the `Avg_m_N`, where N stands for the number os months (looking backwards). For example: 

`Avg_Sep_3` is the average for September, considering 3 months, so we need to accumulate July, August and September to retrieve their mean, and their standar deviation, considering all the years. We have to keep in mind that, when calculating `Avg_Jan_3`, we have to consider November and December from the previous year. 

With all those considerations in mind, let's try to pre-calculate `MONTHLY_AVERAGE_N` and `MONTHLY_STD_N`. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.animation import FuncAnimation

import xarray as xr
import geopandas as gpd

from mergedownloader.file_downloader import FileDownloader, ConnectionType
from mergedownloader.downloader import Downloader
from mergedownloader.inpeparser import *
from mergedownloader.utils import DateProcessor, DateFrequency

In [3]:
fd = FileDownloader(INPE_SERVER, connection_type=ConnectionType.HTTP)

Using wget through HTTP on: ftp.cptec.inpe.br


In [4]:
downloader = Downloader(
    file_downloader=fd,
    parsers=InpeParsers,
    local_folder='/workspaces/reports_folder/downloads2/',
    log_level=logging.DEBUG
)

In [5]:
InpeParsers

{<InpeTypes.DAILY_RAIN: 1>: Instance of DailyParser,
 <InpeTypes.MONTHLY_ACCUM_YEARLY: 2>: Instance of MonthlyAccumYearlyParser,
 <InpeTypes.DAILY_AVERAGE: 3>: Instance of DailyAverageParser,
 <InpeTypes.MONTHLY_ACCUM: 4>: Instance of MonthlyAccumParser,
 <InpeTypes.MONTHLY_ACCUM_MANUAL: 5>: Instance of MonthlyAccumManual,
 <InpeTypes.YEARLY_ACCUM: 6>: Instance of YearAccumulatedParser,
 <InpeTypes.MONTHLY_SP1: 10>: Instance of SPI1Processor,
 <InpeTypes.MONTHLY_AVG_N: 8>: Instance of MonthlyAvgNParser,
 <InpeTypes.MONTHLY_STD_N: 9>: Instance of MonthlyStdNParser}

In [33]:
file = downloader.get_file('2023-03-01', InpeTypes.MONTHLY_ACCUM_MANUAL)

In [34]:
file

PosixPath('/workspaces/reports_folder/downloads2/MONTHLY_ACCUM_MANUAL/MERGE_CPTEC_acum_mar_2023.nc')

In [36]:
dset = xr.open_dataset(file)
dset

<xarray.Dataset>
Dimensions:      (latitude: 924, longitude: 1001, time: 1)
Coordinates:
    step         timedelta64[ns] ...
    surface      float64 ...
  * latitude     (latitude) float64 -60.05 -59.95 -59.85 ... 32.05 32.15 32.25
  * longitude    (longitude) float64 -120.1 -120.0 -119.9 ... -20.15 -20.05
    spatial_ref  int64 ...
  * time         (time) datetime64[ns] 2023-03-01T12:00:00
Data variables:
    pacum        (time, latitude, longitude) float32 ...
Attributes:
    updated:   2024-09-26 14:35:51.594657
    last_day:  20230331
    days:      31

In [40]:
set(dset.attrs).issuperset({'days', 'last_day', 'updated'})

True

In [31]:
arr.rio.crs

CRS.from_epsg(4326)

In [32]:
arr

<xarray.DataArray 'pacum' (time: 1, latitude: 924, longitude: 1001)>
[924924 values with dtype=float32]
Coordinates:
    step         timedelta64[ns] ...
    surface      float64 ...
  * latitude     (latitude) float64 -60.05 -59.95 -59.85 ... 32.05 32.15 32.25
  * longitude    (longitude) float64 -120.1 -120.0 -119.9 ... -20.15 -20.05
    spatial_ref  int64 ...
  * time         (time) datetime64[ns] 2023-03-01T12:00:00

In [29]:
set(arr.dims)

{'latitude', 'longitude'}

In [8]:
cube = downloader.create_cube('2023-03-01', '2023-03-05', InpeTypes.DAILY_RAIN)

/usr/local/lib/python3.10/dist-packages/ecmwflibs/__init__.py:81: UserWarning: libglib-2.0.so.0: cannot open shared object file: No such file or directory
  warnings.warn(str(e))


In [9]:
cube

<xarray.DataArray 'prec' (time: 5, latitude: 924, longitude: 1001)>
array([[[ 0.125 ,  0.125 ,  0.125 , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.125 , ...,  0.    ,  0.    ,  0.    ],
        [ 0.125 ,  0.125 ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        ...,
        [ 0.375 ,  0.125 ,  0.125 , ...,  0.125 ,  0.125 ,  0.125 ],
        [ 0.25  ,  0.25  ,  0.25  , ...,  0.125 ,  0.    ,  0.    ],
        [ 0.375 ,  0.375 ,  0.625 , ...,  0.125 ,  0.    ,  0.    ]],

       [[ 0.    ,  0.    ,  0.    , ...,  0.6875,  0.9375,  0.8125],
        [ 0.    ,  0.    ,  0.    , ...,  0.9375,  0.6875,  0.6875],
        [ 0.    ,  0.    ,  0.    , ...,  0.5   ,  0.625 ,  0.625 ],
        ...,
        [ 3.3125,  4.5625,  4.9375, ...,  0.    ,  0.    ,  0.0625],
        [ 4.25  ,  5.25  ,  5.75  , ...,  0.0625,  0.0625,  0.0625],
        [ 4.0625,  4.25  ,  4.125 , ...,  0.125 ,  0.5   ,  0.375 ]],

       [[ 0.    ,  0.125 ,  0.1875, ...,  8.0625,  7.8125,  8.125 ],
        [ 0.125 ,  0.0625,  0.125 , ...,  7.6875,  7.6875,  7.5   ],
        [ 0.0625,  0.0625,  0.125 , ...,  8.3125,  8.0625,  7.75  ],
        ...,
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.125 ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.375 ,  0.375 ,  0.5625]],

       [[ 0.    ,  5.25  ,  5.5   , ...,  0.    ,  0.    ,  0.    ],
        [ 5.75  ,  5.75  ,  5.5   , ...,  0.    ,  0.    ,  0.    ],
        [ 6.25  ,  5.75  ,  5.625 , ...,  0.    ,  0.    ,  0.    ],
        ...,
        [ 0.25  ,  0.25  ,  0.125 , ...,  0.    ,  0.25  ,  0.375 ],
        [ 0.125 ,  0.125 ,  0.    , ...,  0.875 ,  0.875 ,  0.5   ],
        [ 0.    ,  0.    ,  0.    , ...,  4.375 ,  2.875 ,  2.    ]],

       [[12.25  , 12.75  , 13.75  , ...,  0.    ,  0.    ,  0.    ],
        [19.125 , 18.75  , 14.75  , ...,  0.    ,  0.    ,  0.    ],
        [14.5   , 15.    , 13.5   , ...,  0.    ,  0.    ,  0.    ],
        ...,
        [ 0.375 ,  0.25  ,  0.125 , ...,  1.625 ,  1.875 ,  1.75  ],
        [ 0.75  ,  0.625 ,  0.375 , ...,  2.625 ,  2.125 ,  2.5   ],
        [ 1.    ,  1.    ,  0.625 , ...,  2.125 ,  3.625 ,  3.    ]]],
      dtype=float32)
Coordinates:
  * time         (time) datetime64[ns] 2023-03-01T12:00:00 ... 2023-03-05T12:...
    step         timedelta64[ns] ...
    surface      float64 ...
  * latitude     (latitude) float64 -60.05 -59.95 -59.85 ... 32.05 32.15 32.25
  * longitude    (longitude) float64 -120.1 -120.0 -119.9 ... -20.15 -20.05
    valid_time   datetime64[ns] ...
    spatial_ref  int64 0
Attributes: (12/29)
    GRIB_paramId:                             260138
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      924924
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_name:                                Precipitation
    GRIB_shortName:                           prec
    GRIB_units:                               kg m**-2
    long_name:                                Precipitation
    units:                                    kg m**-2
    standard_name:                            unknown

In [21]:
cube.rio.crs is not None

True

In [18]:
set()

set()

In [16]:
set(cube.dims) == {'time', 'longitude', 'latitude'}

True

In [19]:
len(cube.time)

5

## Testing the file and folder naming convention

In [6]:
avg_parser = InpeParsers[InpeTypes.MONTHLY_AVG_N]
avg_parser.local_folder('2000-01', output_folder='/workspaces/reports_folder/downloads2/', n=3)

PosixPath('/workspaces/reports_folder/downloads2/MONTHLY_STATS_N/MONTHLY_STATS_3')

In [7]:
std_parser = downloader.get_parser('MONTHLY_STD_N')
std_parser.local_target('2000-01', output='/workspaces/reports_folder/downloads2/', n=1)

PosixPath('/workspaces/reports_folder/downloads2/MONTHLY_STATS_N/MONTHLY_STATS_1/Monthly_STD_jan_1.nc')

If we try to grab a file and it does not exists, it should raise an exception, explaining how to calculate them... let's try it.

In [8]:
file = std_parser.local_target('2002-01', output='/workspaces/reports_folder/downloads2/', n=1)
if file.exists():
    file.unlink()

downloader.get_file('2002-01', datatype=InpeTypes.MONTHLY_STD_N, n=1)

ValueError: This type of file MonthlyStdNParser must be pre-computed using the StatsCalculator.Calc_Avg_Std_N() method.

In [21]:
std_parser.__class__.__name__

'MonthlyStdNParser'

In [ ]:
raise RuntimeWarning

## Calculate AVERAGE and STD for N months timeframe 

In [14]:
# First thing we need to calculate the Monthly Average is to create a cube with the `MONTHLY_ACCUM_YEARLY` type.

# The calculation is done on a monthly basis, but, considering we have a moving window, it's easier to 
# grab the entire cube at once

# let's get all the available months into a cube
cube = downloader.create_cube('2000-01', '2023-12', datatype=InpeTypes.MONTHLY_ACCUM_YEARLY)


In [15]:
cube

<xarray.DataArray 'pacum' (time: 283, latitude: 691, longitude: 551)>
array([[[9.187500e+01, 9.275000e+01, 9.675000e+01, ..., 1.350000e+01,
         1.387500e+01, 1.325000e+01],
        [8.912500e+01, 9.587500e+01, 1.011250e+02, ..., 1.587500e+01,
         1.375000e+01, 1.375000e+01],
        [9.587500e+01, 1.000000e+02, 1.003750e+02, ..., 1.512500e+01,
         1.625000e+01, 1.400000e+01],
        ...,
        [2.205000e+02, 2.278750e+02, 2.053750e+02, ..., 6.000000e+00,
         6.250000e+00, 6.750000e+00],
        [1.958750e+02, 1.978750e+02, 2.135000e+02, ..., 8.875000e+00,
         1.062500e+01, 1.037500e+01],
        [1.932500e+02, 1.963750e+02, 2.075000e+02, ..., 1.125000e+01,
         1.325000e+01, 1.275000e+01]],

       [[2.102500e+02, 2.092500e+02, 2.097500e+02, ..., 1.375000e+01,
         1.512500e+01, 1.625000e+01],
        [2.203750e+02, 2.181250e+02, 2.201250e+02, ..., 1.450000e+01,
         1.562500e+01, 1.487500e+01],
        [2.220000e+02, 2.166250e+02, 2.147500e+02, ..., 1.650000e+01,
         1.525000e+01, 1.537500e+01],
...
        [1.953750e+02, 1.790000e+02, 2.038750e+02, ..., 4.500000e+00,
         5.250000e+00, 4.750000e+00],
        [1.896250e+02, 1.687500e+02, 1.898750e+02, ..., 4.625000e+00,
         4.000000e+00, 3.500000e+00],
        [1.743750e+02, 1.752500e+02, 1.728750e+02, ..., 3.875000e+00,
         4.000000e+00, 3.250000e+00]],

       [[1.560625e+02, 1.560625e+02, 1.515000e+02, ..., 1.603125e+02,
         1.541250e+02, 1.576250e+02],
        [1.524375e+02, 1.496875e+02, 1.443750e+02, ..., 1.477500e+02,
         1.460625e+02, 1.408750e+02],
        [1.492500e+02, 1.500000e+02, 1.423750e+02, ..., 1.440000e+02,
         1.426250e+02, 1.420000e+02],
        ...,
        [3.487500e+01, 3.575000e+01, 3.625000e+01, ..., 1.000000e+00,
         6.250000e-01, 6.875000e-01],
        [4.362500e+01, 4.725000e+01, 4.350000e+01, ..., 7.500000e-01,
         2.500000e-01, 6.250000e-01],
        [4.612500e+01, 5.600000e+01, 5.068750e+01, ..., 5.000000e-01,
         3.750000e-01, 6.250000e-01]]])
Coordinates:
  * longitude    (longitude) float64 -85.05 -84.95 -84.85 ... -30.15 -30.05
  * latitude     (latitude) float64 -56.15 -56.05 -55.95 ... 12.65 12.75 12.85
  * time         (time) datetime64[ns] 2000-06-01T12:00:00 ... 2023-12-01T12:...
    spatial_ref  int64 0

In [18]:
moving_average = cube.rolling(time=3).mean()

In [19]:
# remove the dates where we don't have the moving window calculated
moving_average = moving_average.dropna('time', how='all')

In [20]:
moving_average

<xarray.DataArray 'pacum' (time: 281, latitude: 691, longitude: 551)>
array([[[131.125     , 131.29166667, 131.04166667, ...,  14.25      ,
          14.66666667,  14.5       ],
        [132.20833333, 133.41666667, 136.08333333, ...,  14.58333333,
          14.        ,  13.41666667],
        [131.29166667, 133.04166667, 133.79166667, ...,  14.58333333,
          14.70833333,  13.875     ],
        ...,
        [202.33333333, 190.83333333, 181.375     , ..., 100.41666667,
         108.16666667, 111.875     ],
        [197.45833333, 194.29166667, 196.45833333, ...,  90.83333333,
         101.66666667, 117.83333333],
        [204.625     , 201.375     , 202.83333333, ..., 104.5       ,
         125.66666667, 139.16666667]],

       [[121.91666667, 121.25      , 120.29166667, ...,  22.54166667,
          24.08333333,  24.66666667],
        [121.91666667, 122.125     , 122.33333333, ...,  21.08333333,
          21.95833333,  21.33333333],
        [117.66666667, 120.08333333, 119.45833333, ...,  20.54166667,
          20.16666667,  20.04166667],
...
        [118.41666667, 114.5       , 121.45833333, ...,  63.66666667,
          64.66666667,  62.125     ],
        [121.75      , 111.04166667, 111.79166667, ...,  64.375     ,
          61.16666667,  59.41666667],
        [117.04166667, 114.79166667, 111.54166667, ...,  62.70833333,
          57.29166667,  55.45833333]],

       [[181.14583333, 183.10416667, 182.08333333, ..., 126.60416667,
         120.66666667, 118.41666667],
        [180.8125    , 181.64583333, 181.04166667, ..., 117.91666667,
         115.14583333, 113.29166667],
        [183.41666667, 181.75      , 182.91666667, ..., 115.91666667,
         115.33333333, 115.45833333],
        ...,
        [ 95.66666667,  92.375     ,  98.95833333, ...,  25.70833333,
          24.66666667,  23.6875    ],
        [ 96.70833333,  90.125     ,  97.20833333, ...,  23.25      ,
          21.79166667,  21.33333333],
        [ 92.75      ,  94.95833333,  95.97916667, ...,  21.66666667,
          21.25      ,  19.625     ]]])
Coordinates:
  * longitude    (longitude) float64 -85.05 -84.95 -84.85 ... -30.15 -30.05
  * latitude     (latitude) float64 -56.15 -56.05 -55.95 ... 12.65 12.75 12.85
  * time         (time) datetime64[ns] 2000-08-01T12:00:00 ... 2023-12-01T12:...
    spatial_ref  int64 0

In [23]:
grouped_avg = moving_average.groupby('time.month').mean('time')
grouped_std = moving_average.groupby('time.month').std('time')

In [25]:
grouped_avg.isel(month=1)

<xarray.DataArray 'pacum' (latitude: 691, longitude: 551)>
array([[154.06884058, 154.33333333, 154.87952899, ..., 126.70108696,
        128.47373188, 130.66847826],
       [153.66213768, 154.5       , 155.01630435, ..., 126.71376812,
        128.69021739, 130.20289855],
       [154.10597826, 154.48460145, 154.57065217, ..., 126.30887681,
        127.3451087 , 130.18206522],
       ...,
       [ 46.05253623,  45.44655797,  51.54121377, ...,  20.58967391,
         22.30525362,  22.17028986],
       [ 50.24003623,  52.72780797,  57.15398551, ...,  19.13949275,
         19.95289855,  20.2798913 ],
       [ 50.25362319,  55.76268116,  56.71376812, ...,  18.72735507,
         19.22554348,  19.83514493]])
Coordinates:
  * longitude    (longitude) float64 -85.05 -84.95 -84.85 ... -30.15 -30.05
  * latitude     (latitude) float64 -56.15 -56.05 -55.95 ... 12.65 12.75 12.85
    spatial_ref  int64 0
    month        int64 2